<a href="https://colab.research.google.com/github/lustraka/puzzles/blob/main/AoC2021/AoC_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advent of Code Puzzles
[Advent of Code 2021](https://adventofcode.com/2021) | [reddit/adventofcode](https://www.reddit.com/r/adventofcode/)

In [78]:
import requests
import pandas as pd
import numpy as np
import re
path = 'https://raw.githubusercontent.com/lustraka/puzzles/main/AoC2021/data/'

## [Day 13](https://adventofcode.com/2021/day/13): Transparent Origami
### Part I
- **Unknown**: The number of dots visible after completing just the first fold instruction on the transparent paper.
- **Data**: Coordinates of dots and folding instructions
- **Condition**:
  - [0,0] represents the top-left coordinates
  - `x` increases to the right, `y` increases downward
  - `fold along y=...` instruction means folding the paper up
  - `fold along x=xxx` instruction means folding the paper left
- **Plan**:
  - Initialize the matrix
  - Write a function for folding
  - Count the dots after folding


In [79]:
example = """6,10
0,14
9,10
0,3
10,4
4,11
6,0
6,12
4,1
0,13
10,12
3,4
3,0
8,4
1,10
2,14
8,10
9,0

fold along y=7
fold along x=5"""

In [83]:
def parse(input):
  lines = input.split('\n')
  dots = []
  folds = []
  for line in lines:
    if ',' in line:
      dots.append(np.array(line.split(',')).astype(int))
    if '=' in line:
      fold = re.findall('[x,y]=\d+', line)[0].split('=')
      folds.append([fold[0],int(fold[1])])
  return np.array(dots), folds

dots, folds = parse(example)
print(dots[:4])
print(folds)

[[ 6 10]
 [ 0 14]
 [ 9 10]
 [ 0  3]]
[['y', 7], ['x', 5]]


In [84]:
r = requests.get(path+'AoC2021_13.txt')
dots, folds = parse(r.text)
print(dots[-2:])
print(folds[-2:])

[[954 841]
 [960 278]]
[['y', 13], ['y', 6]]


In [85]:
pap = np.zeros(dots.max(axis=0)+1).astype(int)
for dot in dots:
  pap[dot[0], dot[1]] = 1
pap

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

**Beware** of slicing vs. indexing. The expression `pap[6,10]` adresses one element [6,10] in the array, whereas the expression `pap[[6,10]]` addresses two rows [6, :] and [10, :]

In [ ]:
print(dots[0])
print(pap[dots[0]])
print(np.stack((pap[6, :], pap[10, :])))

[ 6 10]
[[1 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]]
[[1 0 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0]]


In [ ]:
# Fold the paper up - indices
y = 7
for i in range(y):
  print(i, 2*y-i)

0 14
1 13
2 12
3 11
4 10
5 9
6 8


In [ ]:
pap_y7 = pap[:,:y].copy()
for i in range(y):
  pap_y7[:,i] += pap[:, 2*y-i]
pap_y7.T

array([[1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 2, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
pap_y7[pap_y7>0].shape

(17,)

In [ ]:
# Fold the paper left
x = 5
pap_x5 = pap_y7[:x,:].copy()
for i in range(x):
  pap_x5[i,:] += pap_y7[2*x-i, :]
pap_x5.T

array([[1, 1, 1, 1, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 0, 0, 0, 1],
       [1, 2, 2, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [ ]:
pap_x5[pap_x5>0].shape

(16,)

In [92]:
def solve_part1(input):
  dots, folds = parse(input)
  # Initialize the paper
  pap = np.zeros(dots.max(axis=0)+1).astype(int)
  for dot in dots:
    pap[dot[0], dot[1]] = 1
  print('Initial shape:', pap.shape)
  
  # Calculate the first fold
  ax, val = folds[0][0], folds[0][1]
  print(ax, val)
  # Fold the paper left
  if ax == 'x':
    for i in range(val):
      pap[i, :] += pap[2*val-i, :]
    # Adjust shape of the paper
    pap = pap[:val, :]
  else:
    for i in range(val):
      pap[:, i] += pap[:, 2*val-i]
    # Adjust shape of the paper
    pap = pap[:, :val]
  print('Folded shape:', pap.shape)

  return pap[pap>0].shape[0]

solve_part1(example)

Initial shape: (11, 15)
y 7
Folded shape: (11, 7)


17

In [93]:
solve_part1(r.text)

Initial shape: (1311, 890)
x 655
Folded shape: (655, 890)


669

### Part II
- **Unknown**: Eight capital letters after folding the paper.
- **Data**: Same as in part I
- **Condition**:
  - Same as in part I
- **Plan**:
  - Calculate all folds and return transposed result


In [96]:
def solve_part2(input):
  dots, folds = parse(input)
  # Initialize the paper
  pap = np.zeros(dots.max(axis=0)+1).astype(int)
  for dot in dots:
    pap[dot[0], dot[1]] = 1
  print('Initial shape:', pap.shape)
  
  # Calculate the folds
  for ax, val in folds:
    #print(ax, val)
    # Fold the paper left
    if ax == 'x':
      for i in range(val):
        pap[i, :] += pap[2*val-i, :]
      # Adjust shape of the paper
      pap = pap[:val, :]
    else:
      for i in range(val):
        pap[:, i] += pap[:, 2*val-i]
      # Adjust shape of the paper
      pap = pap[:, :val]
    #print('Folded shape:', pap.shape)

  return pap.T

code = solve_part2(example)
print(code)

Initial shape: (11, 15)
[[1 1 1 1 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 0 0 0 1]
 [1 2 2 1 1]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [97]:
solve_part2(r.text)

Initial shape: (1311, 890)


IndexError: ignored